In [1]:
#Import Dependencies
import numpy as np
import pandas as pd

In [2]:
#Read the percent_stats csv file into a pandas DataFrame
percent_stats_df = pd.read_csv('percent_stats.csv')
percent_stats_df.head()

,Unnamed: 0,Year,Pos,TRB%,AST%,STL%,BLK%,TOV%,USG%,3P%,2P%,FT%
0,0,1982.0,C,13.4,11.9,1.1,4.1,13.3,25.6,0.000,0.580,0.706
1,1,1982.0,C,13.6,22.1,2.3,1.9,14.8,22.8,0.000,0.494,0.781
2,2,1982.0,SF,9.7,18.6,1.2,0.9,12.7,29.8,0.352,0.475,0.680
3,3,1982.0,SG,5.5,19.7,3.1,0.3,17.5,21.5,0.294,0.363,0.862
4,4,1982.0,PG,2.9,31.9,1.1,0.1,18.4,17.9,0.375,0.475,0.747


In [41]:
#Read the raw_stats csv file into a pandas DataFrame
raw_stats_df = pd.read_csv('raw_stats.csv')
raw_stats_df.head()

,Unnamed: 0,Year,Pos,G,MP,FG,FGA,3P,3PA,2P,2PA,FT,FTA,TRB,AST,STL,BLK,TOV,PF,PTS
0,0,1982.0,C,76.0,2677.0,753.0,1301.0,0.0,3.0,753.0,1298.0,312.0,442.0,659.0,225.0,63.0,207.0,230.0,224.0,1818.0
1,1,1982.0,C,79.0,2393.0,507.0,1027.0,0.0,1.0,507.0,1026.0,182.0,233.0,586.0,356.0,114.0,78.0,196.0,269.0,1196.0
2,2,1982.0,SF,51.0,1468.0,381.0,820.0,25.0,71.0,356.0,749.0,168.0,247.0,249.0,164.0,37.0,22.0,135.0,152.0,955.0
3,3,1982.0,SG,53.0,564.0,79.0,221.0,5.0,17.0,74.0,204.0,56.0,65.0,56.0,87.0,37.0,3.0,53.0,86.0,219.0
4,4,1982.0,PG,68.0,2167.0,308.0,652.0,6.0,16.0,302.0,636.0,236.0,316.0,116.0,541.0,52.0,3.0,178.0,131.0,858.0


In [4]:
#Convert the string value for the "Pos" columns into a usable data type
# Step 0: Reformat data
data = percent_stats_df.values
y = data[:, 2]

from sklearn.preprocessing import LabelEncoder
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)
    
from keras.utils import to_categorical
# Step 2: One-hot encoding
one_hot_y = to_categorical(encoded_y)
one_hot_y

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [60]:
#Define the selected features for the percentage model
percent_selected_features = ['TRB%','AST%','BLK%','TOV%','3P%','USG%']

#Define X and y sets
X = percent_stats_df[percent_selected_features]
y = one_hot_y

#Split data into training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, data[:, 2], random_state=0)

#Print the X_train dataframe
X_train.head()

,TRB%,AST%,BLK%,TOV%,3P%,USG%
11173,11.0,7.7,0.9,10.4,0.375,13.1
9174,5.7,15.1,0.3,16.6,0.154,22.0
3412,14.2,4.6,1.4,12.8,0.000,22.5
5939,2.7,22.3,0.0,63.6,0.500,23.4
4719,3.3,37.0,0.0,33.3,1.000,7.5


In [61]:
#Import the StandardScaler for scaling the dataset
from sklearn.preprocessing import StandardScaler

#Generate the scaling function for the features
X_scaler = StandardScaler().fit(X_train)

#Apply the scaling function to the features
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [62]:
# Create the SVC Model
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model

SVC(kernel='linear')

In [63]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10],
              'gamma': [0.0001, 0.001, 0.01]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [64]:
# Fit the model using the grid search estimator. 
# This will take the SVC model and try each combination of parameters
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................... C=1, gamma=0.0001, score=0.604, total=   6.7s
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.7s remaining:    0.0s


[CV] ................... C=1, gamma=0.0001, score=0.616, total=   5.5s
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   12.2s remaining:    0.0s


[CV] ................... C=1, gamma=0.0001, score=0.627, total=   5.5s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.616, total=   4.9s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.626, total=   4.8s
[CV] C=1, gamma=0.001 ................................................
[CV] .................... C=1, gamma=0.001, score=0.604, total=   4.9s
[CV] C=1, gamma=0.001 ................................................
[CV] .................... C=1, gamma=0.001, score=0.616, total=   6.3s
[CV] C=1, gamma=0.001 ................................................
[CV] .................... C=1, gamma=0.001, score=0.627, total=   5.8s
[CV] C=1, gamma=0.001 ................................................
[CV] .................... C=1, gamma=0.001, score=0.616, total=   4.8s
[CV] C=1, gamma=0.001 ................................................
[CV] .

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  7.4min finished


GridSearchCV(estimator=SVC(kernel='linear'),
             param_grid={'C': [1, 5, 10], 'gamma': [0.0001, 0.001, 0.01]},
             verbose=3)

In [65]:
# List the best parameters for this dataset
print(grid.best_params_)

{'C': 5, 'gamma': 0.0001}


In [66]:
# List the best score
print(grid.best_score_)

0.6179559118236473


In [67]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test_scaled)

In [68]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,
                            target_names=["C", "PF", "SF", "SG", "PG"]))

              precision    recall  f1-score   support

           C       0.66      0.62      0.64       840
          PF       0.53      0.54      0.54       900
          SF       0.83      0.82      0.82       849
          SG       0.53      0.50      0.51       775
          PG       0.57      0.64      0.60       795

    accuracy                           0.62      4159
   macro avg       0.62      0.62      0.62      4159
weighted avg       0.63      0.62      0.62      4159

